## config & startup

1. 修改docker-compose.yaml中的用户名和密码，控制台输入 `docker-compose up -d` 运行后台mongodb.
2. 登录`http://localhost:8081/`

In [1]:
from mongodb_helper import *

# mongodb config
mg_host = "localhost"
mg_port = 27017
mg_user = "root"
mg_password = "admin"

cn_stock = "cn_stock"

## cn_stock/trade_cal

### Overwrite

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("datasources/trade_cal.csv", parse_dates=['dt'], date_format='%Y%m%d', dtype={'is_open': np.uint8})
df = df.rename(columns={'dt': '_id', 'is_open': 'status'}) # mongodb要求必须有_id
# print(df.info())

with mongo_connect(mg_host, mg_port, mg_user, mg_password) as mg:
    db = mg[cn_stock]
    db.drop_collection('trade_cal')
    db['trade_cal'].insert_many(df.to_dict(orient='records'))

### Read

In [4]:
import pandas as pd
import quantdata as qd

with qd.open_mongodb(host=mg_host, port=mg_port, user=mg_user, password=mg_password):
    c = qd.mongo_get_trade_cal(cn_stock)
    df = pd.DataFrame(c)
df

connecting mongodb...
mongodb closing...
mongodb closed


,_id,status
0,1990-12-19,1
1,1990-12-20,1
2,1990-12-21,1
3,1990-12-24,1
4,1990-12-25,1
...,...,...
8307,2024-12-25,1
8308,2024-12-26,1
8309,2024-12-27,1
8310,2024-12-30,1


## cn_stock/stocks_basic_info

### Overwrite

In [5]:
import pandas as pd

df = pd.read_csv("datasources/stocks_basic_info.csv", parse_dates=['list_date', 'delist_date'], date_format='%Y%m%d', index_col=0)
df = df.drop(columns=['symbol'])
df = df.rename(columns={'ts_code':'symbol', 'list_status':'status'})
df = df.fillna('')
# print(df.info())

with mongo_connect(mg_host, mg_port, mg_user, mg_password) as mg:
    db = mg[cn_stock]
    db.drop_collection('stocks_basic_info')
    insert_many_ignore_nan(db['stocks_basic_info'], df)

### Read

In [6]:
import pandas as pd
import quantdata as qd

with qd.open_mongodb(host=mg_host, port=mg_port, user=mg_user, password=mg_password):
    c = qd.mongo_get_data(cn_stock, 'stocks_basic_info')
    df = pd.DataFrame(c)
df

connecting mongodb...
mongodb closing...
mongodb closed


,_id,symbol,name,area,industry,fullname,enname,cnspell,market,exchange,curr_type,status,list_date,is_hs,delist_date
0,6784807106bc46911780b6d9,000001.SZ,平安银行,深圳,银行,平安银行股份有限公司,"Ping An Bank Co., Ltd.",payh,主板,SZSE,CNY,L,1991-04-03,S,NaT
1,6784807106bc46911780b6da,000002.SZ,万科A,深圳,全国地产,万科企业股份有限公司,"China Vanke Co.,Ltd.",wka,主板,SZSE,CNY,L,1991-01-29,S,NaT
2,6784807106bc46911780b6db,000004.SZ,国华网安,深圳,软件服务,深圳国华网安科技股份有限公司,Shenzhen Guohua Network Security Technology Co...,ghwa,主板,SZSE,CNY,L,1991-01-14,N,NaT
3,6784807106bc46911780b6dc,000006.SZ,深振业A,深圳,区域地产,深圳市振业(集团)股份有限公司,"Shenzhen Zhenye(Group) Co., Ltd.",szya,主板,SZSE,CNY,L,1992-04-27,S,NaT
4,6784807106bc46911780b6dd,000007.SZ,全新好,深圳,其他商业,深圳市全新好股份有限公司,"Shenzhen Quanxinhao Co,.Ltd.",qxh,主板,SZSE,CNY,L,1992-04-13,N,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5621,6784807106bc46911780ccce,688555.SH,退泽达(退),,,泽达易盛(天津)科技股份有限公司,"Essence Information Technology Co., Ltd.",tszd,科创板,SSE,CNY,D,2020-06-23,N,2023-07-07
5622,6784807106bc46911780cccf,832317.BJ,观典防务(退),,,观典防务技术股份有限公司,"Guandian Defense Technology Co.,Ltd.",gdfw,北交所,BSE,CNY,D,2020-07-27,N,2022-04-26
5623,6784807106bc46911780ccd0,833874.BJ,泰祥股份(退),,,十堰市泰祥实业股份有限公司,"Shiyan Taixiang Industry Co., Ltd.",txgf,北交所,BSE,CNY,D,2020-07-27,N,2022-07-18
5624,6784807106bc46911780ccd1,833994.BJ,翰博高新(退),,,翰博高新材料(合肥)股份有限公司,"Highbroad Advanced Material (Hefei) Co., Ltd.",hbgx,北交所,BSE,CNY,D,2020-07-27,N,2022-07-25
